# How to do single-cell analysis of ISS data:

This notebook guides you through the different steps that will eventually allow you to analyse ISS data at the single cell level.

To complete this tutorial you should have completed the previous steps, and you will need the following ingredients.

1. Your decoded data, in the standard decoded format as we have generated from the `ISS_decoding` notebook.
2. A segmentation mask in .nzp format produced with either `Stardist`, `Cellpose` (see our tutorials) or whichever method you prefer.
3. time, patience, and some knowledge of single-cell analysis.


The analysis is based on Scanpy (documentation in  https://scanpy.readthedocs.io/en/stable/index.html) and Squidpy (documentation in https://squidpy.readthedocs.io/en/stable/)


## Import packages
We import the necessary packages. `ISS_postprocessing` includes some modules requiring a functional and properly installed CUDA-compatible GPU. They will work but they might be slower on CPU.

In [ ]:
import ISS_postprocessing
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt

In the following code block you need to add the paths to each one of the samples you want to post-process. Note that you can post-process multiple samples in one go, but they will all be analysed in the same way. This is normally OK, but there might be exceptions in which you want to treat your samples differently for some reason.



In [ ]:
samples = ['/home/marco/Downloads/media/marco/mountstuff/rfl_test/']

## Integrate the ISS data on the segmentation mask and create AnnData objects

In this step you are taking your decoded spots and assign each spot a specific cell based on its x,y positions, using a pre-computed segmentation mask to determine the cell boundaries. 

Then the code implements some filtering and conversion steps, and save the object as an `Annotated_Data`.

Let's look at the variables:


`spots_file` = the location of the decoded spots .csv file 

`segmentation_mask`= the location to the npz expanded segmentation mask

`output_file` = the location where the hda5 AnnData file will be saved for each sample

`filter_data`= A boolean specifying whether you want to filter the ISS data on a quality criterion (default=True) 

`metric` = defines **which parameter you want to use for filtering the ISS data** I suggest using the `quality_minimum`

`write_h5ad` = A boolean specifying if the AnnData needs to be saved as h5ad (default==True)

`value`= this is a value specifying **which value is used for filterin** (if quality_minimum was chosen as metric, I'd suggest 0.4-0.5)

`convert_coords` = This converts the coordinates from pixel to nm, and whether it's needed or not will depend on your settings in the `ISS_decoding` notebook. default is True 

`conversion_factor` = this specifies the XY pixel size.

These last 2 steps are to ensure your decoded data actually match the DAPI image. If you don't do these steps right, your segmentation will be completely off and your data will not make any sense.

In [ ]:
for sample_path in samples:  
        ad = ISS_postprocessing.annotated_objects.create_anndata_obj(spots_file = sample_path+'/decoded_dense/decoded.csv', 
            segmentation_mask = sample_path+'cellpose_expanded.npz',#'cell_segmentation/cellpose_segmentation_expanded_2.npz' 
            output_file = sample_path+'/decoded_dense/anndata_cellpose.h5ad',
            filter_data=True, 
            metric = 'quality_minimum', 
            write_h5ad = True,
            value=  0.4,
            convert_coords = False, 
            conversion_factor = 0.1625)

## Concatenation of multiple AnnData objects for joint analysis.

When working with multiple samples, you might want to concatenate all the respective  AnnData objects into one single file. This allows you to treat all of your sections as a single dataset. The code block down here formats the data in a way that `.obs['sample_id']` will contain data about a specific section.

In [ ]:
ad =  ISS_postprocessing.annotated_objects.concat_anndata(sample_anndata_list = samples, 
                  anndata_name = 'anndata_stardist.h5ad'
                  )
ad.obs['sample_id'] = ad.obs['sample_id'].str.split('/', expand = True)[5] 

In the following step some useful metrics about the experiment are calculated. 

In [ ]:
#import scanpy as sc
#ad=sc.read('/home/marco/Downloads/media/marco/mountstuff/rfl_test/decoded_dense/anndata_stardist.h5ad')

In [ ]:
sc.pp.calculate_qc_metrics(ad, percent_top=None, log1p=False, inplace=True)

The following block of code allows us to visualize the distribution of `total_counts` in the cells. This will allow you to determine a few things, and understand your data a bit.

1. cells with a very low `total_counts` will be useless for clustering (especially 0s) so you might want to filter them out

2. cells with unusually high `total_counts` are often arising because of segmentation artifacts. If most of the cells have around 30 counts, cells with 100s of counts are somewhat suspicious.

In [ ]:
import matplotlib.pyplot as plt
ad.obs['total_counts'].plot(kind='box')
plt.title('Boxplot of total_counts')
plt.show()

Similar considerations apply also to this other metrics, the `n_genes_by_counts` measure. This tells us **how many unique genes are detected in each cell**. Cells that express only 1 or very few genes are usually suspicious, and we might want to filter them out.

**All of these considerations of course will depend on the number of genes you probe for in the ISS panel and also the efficiency of detection, etc...**
The boxplots above are meant to visually inspect these metrics and aid in making a decision about meaningful filtering threshold in the next step.

In [ ]:
import matplotlib.pyplot as plt
ad.obs['n_genes_by_counts'].plot(kind='box')
plt.title('Boxplot of n_genes_by_counts')
plt.show()

## Set filtering thresholds and discard uninformative cells

By modifying the line below we get to choose values to filter the cells based on the metric discussed before:

In the example here, we'll keep only the cells with more than 5 spots, coming from at least 2 different genes. 

In [ ]:
# Step 1: Convert cell index to string
ad.obs_names = ad.obs_names.astype(str)
# Step 2: Establish filtering criteria
mask = (ad.obs['total_counts'] > 5) & (ad.obs['n_genes_by_counts'] > 2)
# Step 3: Establish filtering criteria
ad = ad[mask].copy()


In [ ]:
len(ad)

## Normalization and transformation

Now your data is stored in the `ad` object as raw data. 

We now copy the raw data into `ad.raw`, and store it for later use if needed. 

To the data in `ad` we apply normalization and log-transformation to minimize the effect of outliers. 

We also scale the data to give the same importance to all genes in clustering.

There's a lot of debate about which normalization method one should use for ISS data, and whether one would need to scale at all, so take all of this with a grain of salt.



In [ ]:
ad.layers["raw"] = ad.X.copy()
sc.pp.normalize_total(ad)
sc.pp.log1p(ad)

Here we run a standard PCA and proceed with the analysis as in scRNAseq workflows.

In [ ]:
sc.tl.pca(ad, svd_solver='arpack')
plt.rcParams['figure.facecolor'] = 'white'
sc.pl.pca_variance_ratio(ad, log=True)

By changing the `n_neighbors`, `n_pcs` and `min_dist` in the code below, you can change the clustering properties. Refer to the `scanpy` manual for more information.

In [ ]:
sc.pp.neighbors(ad, n_neighbors=20, n_pcs=40)
sc.tl.umap(ad,min_dist=0.005) #0.005

Here we choose 2 resolutions to visualize the cells on the UMAP and plot the results.

In [ ]:
for i in [1, 2]:
    if "norm_leiden_"+str(i) in ad.obs.columns:
        plt.rcdefaults()
        with plt.rc_context({'figure.figsize': (15, 10)}):
            sc.pl.umap(ad, color = ("norm_leiden_"+str(i)),s=20,add_outline=False,legend_loc='on data',legend_fontsize=20,legend_fontoutline=2,alpha = 1)
    else: 
        print('clustering @ resolution ' + str(i))
        sc.tl.leiden(ad, resolution =i, key_added = ("norm_leiden_"+str(i)))
        plt.rcdefaults()
        with plt.rc_context({'figure.figsize': (15, 10)}):
            sc.pl.umap(ad, color = ("norm_leiden_"+str(i)),s=20,add_outline=False,legend_loc='on data',legend_fontsize=20,legend_fontoutline=2)



This block of code extracts the positions of the cells for plotting.


In [ ]:
spatial = np.array(ad.obs[['x','y']].astype('<f8'))
ad.obsm['spatial'] = spatial

ad.obsm['xy_loc'] = spatial

And finally here we can plot the **spatial distribution of the different clusters** in one go.

In [ ]:
cluster = 'norm_leiden_2'
for sample in ad.obs['sample_id'].unique():
    print(sample)
    ad_int = ad[ad.obs['sample_id'] == sample]    
    with plt.rc_context({'figure.figsize': (25, 17.5)}):
        sc.pl.spatial(ad_int,color=cluster ,spot_size=30)




In [ ]:
cluster = 'norm_leiden_1'
ad_int = ad   
with plt.rc_context({'figure.figsize': (25, 17.5)}):
    sc.pl.spatial(ad_int,color=cluster ,spot_size=30)

## Which genes are good markers of your ISS clusters?
By running the following code blocks, you can see which genes characterise each one of the ISS clusters inferred above, and generate different plots showing essentially the same information in less or more condensed format.

In [ ]:
sc.tl.rank_genes_groups(ad, cluster, method='wilcoxon', key_added = cluster+ "_wilcoxon")
sc.pl.rank_genes_groups(ad, n_genes=15, sharey=False, key=cluster+ "_wilcoxon")

In [ ]:
plt.rcdefaults()
sc.pl.rank_genes_groups_matrixplot(ad, n_genes=3, key=cluster+ "_wilcoxon", groupby=cluster, cmap = 'viridis', )

## The following code allows to to save an updated version of the anndata object including all the calculations above.

By running the following code, you are saving an anndata files that includes:

- UMAP representations
- Differentially Expressed Genes by clusters
- Spatial coordinates of the cells

In [ ]:
ad.write("/home/marco/Downloads/media/marco/mountstuff/rfl_test/decoded_dense/anndata_cellpose.h5ad")

## The following code allows to generate figures for each cluster individually.

By running the following code, you plot each cluster, one by one, for all the samples in a single figure, together with a short list of unique markers.



In [ ]:
ad.obs['project'] = 'sc'

In [ ]:
for cluster_ in sorted(ad.obs[cluster].value_counts().head(5).index.astype(int)):
    print(cluster_)

    
    genes = list(sc.get.rank_genes_groups_df(ad,group=str(cluster_), key='wilcoxon')['names'].head(4))
    genes.insert(0, cluster)
    genes.insert(1, 'sample_id')
    plt.rcdefaults()
    sc.pl.umap(ad, color = genes, cmap = 'turbo', ncols = 3, legend_loc='on data',legend_fontsize=10,legend_fontoutline=2)
    
    ISS_postprocessing.annotated_objects.plot_specific_cluster(ad,
                    clusters_to_map = cluster,
                    cluster = str(cluster_),
                    broad_cluster = 'project',
                    key='wilcoxon',
                    size=0.5,
                    number_of_marker_genes=10,
                    sample_id_column='sample_id',
                    dim_subplots=[4, 4],)
    plt.show()